# 独立モデル

$\chi^2$検定における独立モデル（Baseline Model）の自由度がsemopyとlavaanで一致しないことがある。

どういうことなのか？

## （確認）$\chi^2$検定と自由度

最尤法で母数を推定した場合、自由度

$$
df = \frac{1}{2} p (p + 1) - q
$$

の$\chi^2$分布に近似的に従う統計量が得られるので、これを用いて検定を行うことができる（豊田 2012）

ここで$p$は観測変数の数、$q$は自由母数の数

$\chi^2$検定自体はサンプルサイズへの依存性が高くて不正確だが、CFIなど他の指標の算出に用いられる。


### semopyの自由度の計算


Pythonのsemopyパッケージでの自由度の推定方法も豊田同様

```python
def calc_dof(model):
    p = len(model.vars['observed'])
    return p * (p + 1) // 2 - len(model.param_vals)
```

ref.: https://gitlab.com/georgy.m/semopy/-/blob/fa9f3e0bd2bdacac377c8ee339255b27f4ca22f8/semopy/stats.py#L187

### lavaanの自由度の推定

Rの{lavaan}パッケージでの自由度の推定。こちらも概ね同様な様子

[ソースコード](https://github.com/yrosseel/lavaan/blob/6f047c800206d23f246d484b9522295257614222/R/lav_partable_utils.R#L242-L256)を見ると

```R
# degrees of freedom
df <- ndat - npar
```

`npar`は（自由母数の数 ["total number of free parameters"](https://github.com/yrosseel/lavaan/blob/6f047c800206d23f246d484b9522295257614222/R/lav_partable_utils.R#L234-L235)）

`ndat`は（ブロックごとの標本母数の数 ["number of sample statistics per block"](https://github.com/yrosseel/lavaan/blob/6f047c800206d23f246d484b9522295257614222/R/lav_partable_utils.R#L109-L110)）

つまり

$$
標本母数 - 自由母数
$$

とのこと。標本母数のコードは複雑すぎてよくわからなかったが、いくつかのモデルで確認した感じでは基本的に $p (p + 1) / 2$ と一致する

## 独立モデル

**独立モデル（independent model, baseline model, null model)** は、すべての観測変数間に一切のパスを引かず、各変数の分散のみを自由推定するモデルのこと

> すべての観測変数間に一切のパスを引かず、各変数の分散のみを自由推定するモデルを独立モデルと考えることが一般的です。 （豊田 2014, p.193）

### semopy

`get_baseline_model()`で具体的なモデルの状況を確認することができる

```python
from semopy.stats import get_baseline_model
baseline = get_baseline_model(model)
baseline.inspect()
```

### lavaan

独立モデルは`lav_partable_independence()`（[source](https://github.com/yrosseel/lavaan/blob/a271f07a212fcf2902352833a37bde2202eb6b00/R/lav_partable_unrestricted.R#L34)）で取得できる

```r
lav <- lav_partable_independence(fit)
data.frame(lav)
```

また、`lav_partable_npar()`で自由母数の数を調べることができる

```r
# 自由母数の数
lav_partable_npar(lav)
```


### 両者の差異

一部のモデルでは、**lavaanの独立モデルでは共分散も自由母数として推定される**。
そのため、semopyと一致しないことがある。


### 関連するlavaanのメーリングリストのスレッド



:::{card}
[Degrees of Freedom Calculations](https://groups.google.com/g/lavaan/c/zqjDtUx_b9c)

> Ed Rigdon has proposed an even more conservative option that fixes the covariances to the average covariance.

として、[Rigdon, E. E. (1998). The equal correlation baseline model for comparative fit assessment in structural equation modeling.](https://www.tandfonline.com/doi/abs/10.1080/10705519809540089)が紹介されている。

議論もあったが、より保守的なので採用したっぽい

> The choice is just a matter of convention but the consensus was that this practice inflates comparative fit indices and that free means offered more conservative values. 

これは共分散を自由母数にするわけではないが、変数間の共分散を均一にするというアイデアらしい

>  It is better described as one that constrains all the covariances to be equal to one another.  Here is the reference.
> https://www.tandfonline.com/doi/abs/10.1080/10705519809540089
:::

:::{card}
[Null/baseline model](https://groups.google.com/g/lavaan/c/iX2usbOxLlg)

> For continuous data, the default null model in lavaan (and all other SEM software I am aware of) is the "independence" model, which constraints all covariances to zero, and only freely estimates means and variances.
:::

## 参考文献

- 豊田秀樹（2012）『因子分析入門』
- 豊田秀樹（2014）『共分散構造分析［Ｒ編］―構造方程式モデリング―』